## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [156]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [157]:
import platform
from matplotlib import rc

# 운영 체제에 따라 폰트 설정
if platform.system() == 'Windows':  # Windows 환경
    rc('font', family='Malgun Gothic')  # 맑은 고딕

# 음수 표시 가능하도록 설정
plt.rcParams['axes.unicode_minus'] = False

In [158]:
# 경고문구 숨기기
import warnings
import pandas as pd

warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

반복적으로 쓰는 함수툴

In [159]:
def plot_box(df, col_name):
    """
    주어진 데이터프레임과 열 이름에 대해 박스 플롯을 그리는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    col_name (str): 열 이름
    """
    plt.figure(figsize=(7, 4))
    plt.boxplot(df[col_name], vert=False)
    plt.xlabel(col_name)
    plt.title(f'Box Plot of {col_name}')
    plt.show()

def value_counts_ratio(df, col_name, target_name='임신_성공_여부'):
    """
    주어진 데이터프레임의 특정 열에 대해 각 값마다 타겟 변수의 비율과 갯수, 총 갯수를 출력하는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    col_name (str): 열 이름
    target_name (str): 타겟 변수 이름
    """
    # 각 값마다 타겟 변수의 비율 계산
    value_counts = df.groupby(col_name)[target_name].value_counts(normalize=True).unstack().fillna(0)
    
    # 각 값마다 타겟 변수의 갯수 계산
    counts = df.groupby(col_name)[target_name].value_counts().unstack().fillna(0)
    
    # 각 값마다 총 갯수 계산
    total_counts = df[col_name].value_counts().rename('Total_Count')
    
    # 비율과 갯수를 합침
    result = value_counts.join(counts, lsuffix='_ratio', rsuffix='_count')
    
    # 총 갯수를 합침
    result = result.join(total_counts, on=col_name)
    
    # 출력 형식 조정
    result.index.name = 'variable'
    print(f"\n{col_name}별 {target_name} 비율 및 갯수\n")
    print(result.rename(columns=lambda x: x.split('_')[0]))

def summarize_group(df, group_by_columns):
    """
    주어진 데이터프레임을 특정 열들로 그룹화하여 각 그룹의 '임신_성공_여부' 비율과 갯수를 요약하는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    group_by_columns (list): 그룹화할 열 이름들의 리스트
    """
    # 데이터프레임을 그룹화
    grouped_df = df.groupby(group_by_columns)
    
    # 결과를 저장할 리스트 초기화
    results = []
    
    # 그룹화된 데이터프레임의 내용을 확인하는 코드
    for name, group in grouped_df:
        # 그룹의 갯수 계산
        group_count = group.shape[0]
        
        # '임신_성공_여부' 변수의 '임신 성공' 비율과 갯수 계산
        success_count = group['임신_성공_여부'].value_counts().get(1, 0)
        success_ratio = success_count / group_count
        
        # 결과 리스트에 추가
        results.append([name, success_count, success_ratio, group_count])
    
    # 결과 리스트를 데이터프레임으로 변환
    results_df = pd.DataFrame(results, columns=['group', "'임신_성공_여부' count", 'ratio', 'Total'])
    
    # 그룹화된 변수들의 이름을 제목행으로 출력
    print(f"Grouped by: {', '.join(group_by_columns)}")
    print()
    # 데이터프레임 출력
    print(results_df)

def plot_ratio(df, group_by_column, target_column='임신_성공_여부', success_value=1):
    """
    주어진 데이터프레임을 특정 열로 그룹화하여 각 그룹의 타겟 변수의 성공 비율을 시각화하는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    group_by_column (str): 그룹화할 열 이름
    target_column (str): 타겟 변수 이름
    success_value (int): 타겟 변수의 성공을 나타내는 값
    """
    # 데이터프레임을 그룹화
    grouped_df = df.groupby(group_by_column)
    
    # 결과를 저장할 리스트 초기화
    results = []
    
    # 그룹화된 데이터프레임의 내용을 확인하는 코드
    for name, group in grouped_df:
        # 그룹의 갯수 계산
        group_count = group.shape[0]
        
        # '임신_성공_여부' 변수의 '성공' 비율과 갯수 계산
        success_count = group[target_column].value_counts().get(success_value, 0)
        success_ratio = success_count / group_count
        
        # 결과 리스트에 추가
        results.append([name, success_count, success_ratio, group_count])
    
    # 결과 리스트를 데이터프레임으로 변환
    results_df = pd.DataFrame(results, columns=['group', f"'{success_value}' count", 'ratio', 'Total'])
    
    # 그래프 크기 설정
    plt.figure(figsize=(10, 5))
    
    # 막대 그래프 생성
    ax = results_df.plot(kind='bar', x='group', y='ratio', legend=False)
    
    # 각 막대 위에 임신 성공 여부 갯수와 총 갯수 표시
    for i, (success_count, total) in enumerate(zip(results_df[f"'{success_value}' count"], results_df['Total'])):
        ax.text(i, results_df['ratio'][i], f'{success_count} ({total})', ha='center', va='bottom', fontsize=8)
    
    # 그래프 제목 및 축 레이블 설정
    ax.set_title(f'{success_value} Ratio by {group_by_column}')
    ax.set_xlabel(group_by_column)
    ax.set_ylabel(f'{success_value} Ratio')
   
    # 그래프 출력
    plt.show()

def plot_ratio_boxplot(data, column_name, target_column='임신_성공_여부'):
    """
    주어진 데이터프레임의 특정 열과 타겟 변수 간의 관계를 박스 플롯으로 시각화하는 함수.

    Parameters:
    data (pd.DataFrame): 데이터프레임
    column_name (str): 열 이름
    target_column (str): 타겟 변수 이름
    """

    # 그래프 그리기
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=column_name, y=target_column, data=data)

    # 그래프 제목 및 레이블 설정
    plt.title(f'{column_name} vs {target_column}')
    plt.xlabel(column_name)
    plt.ylabel(target_column)

    # 그래프 출력
    plt.show()

### Data Load

In [160]:
# 데이터 로드
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

# 변수명에서 공백을 밑줄로 대체
train.columns = train.columns.str.replace(' ', '_')
test.columns = test.columns.str.replace(' ', '_')

In [161]:
# train.info(verbose=True)

## EDA

In [162]:
# '시술_유형' 변수의 값이 'IVF'에 해당 안하는 데이터 추출
IVF_train = train[train['시술_유형'] == 'IVF'] 
IVF_test = test[test['시술_유형'] == 'IVF']

# 모든 값이 결측값인 변수를 제거
IVF_train = IVF_train.dropna(axis=1, how='all')
IVF_test = IVF_test.dropna(axis=1, how='all')

# 결측값을 제외한 고유값이 1개인 변수 제거
unique_value_columns_train = IVF_train.columns[IVF_train.nunique(dropna=True) <= 1]
unique_value_columns_test = IVF_test.columns[IVF_test.nunique(dropna=True) <= 1]

# 결측값이 없는 상태에서 고유값이 1개인 경우만 제거
IVF_train = IVF_train.drop(columns=[col for col in unique_value_columns_train if IVF_train[col].notna().all()])
IVF_test = IVF_test.drop(columns=[col for col in unique_value_columns_test if IVF_test[col].notna().all()])

# 결과를 원래 변수에 저장
train = IVF_train
test = IVF_test

타겟변수 비율 확인

In [163]:
# '임신_성공_여부' 변수의 값 개수 계산
value_counts = train['임신_성공_여부'].value_counts()

# 각 값이 전체에서 차지하는 비율 계산
value_percentages = value_counts / len(train) * 100

# 결과 출력
print("임신_성공_여부 값 개수:")
print(value_counts)
print("\n임신_성공_여부 값 비율 (%):")
print(value_percentages)

임신_성공_여부 값 개수:
임신_성공_여부
0    184643
1     65417
Name: count, dtype: int64

임신_성공_여부 값 비율 (%):
임신_성공_여부
0    73.839479
1    26.160521
Name: count, dtype: float64


### 1. 시술_시기_코드

난임 시술을 받은 시기를 기준으로 코드 부여

In [164]:
value_counts_ratio(train, '시술_시기_코드')


시술_시기_코드별 임신_성공_여부 비율 및 갯수

                 0         1      0     1  Total
variable                                        
TRCMWS    0.739895  0.260105  27567  9691  37258
TRDQAZ    0.751191  0.248809  28549  9456  38005
TRJXFG    0.730081  0.269919  25693  9499  35192
TRVNRY    0.736404  0.263596  25917  9277  35194
TRXQMD    0.741403  0.258597  25161  8776  33937
TRYBLT    0.727689  0.272311  26092  9764  35856
TRZKPL    0.741348  0.258652  25664  8954  34618


In [165]:
# '시술_시기_코드' 변수 제거
train = train.drop(columns=['시술_시기_코드'])
test = test.drop(columns=['시술_시기_코드'])

### 2. 시술 당시 나이

환자의 시술 당시 나이(연령대)

In [166]:
len(train)

250060

In [167]:
value_counts_ratio(train, '시술_당시_나이')


시술_당시_나이별 임신_성공_여부 비율 및 갯수

                 0         1        0        1   Total
variable                                              
만18-34세   0.674737  0.325263  67747.0  32658.0  100405
만35-37세   0.718321  0.281679  40463.0  15867.0   56330
만38-39세   0.779704  0.220296  29780.0   8414.0   38194
만40-42세   0.838043  0.161957  30431.0   5881.0   36312
만43-44세   0.878486  0.121514  10396.0   1438.0   11834
만45-50세   0.825871  0.174129   5497.0   1159.0    6656
알 수 없음    1.000000  0.000000    329.0      0.0     329


In [168]:
# 필요한 라이브러리 임포트
import pandas as pd

# '시술_당시_나이' 변수의 값이 '알 수 없음'인 행 제거
train = train[train['시술_당시_나이'] != '알 수 없음']
test = test[test['시술_당시_나이'] != '알 수 없음']

In [169]:
# '시술_당시_나이' 변수의 값을 중앙값으로 변환하는 함수 정의
def convert_age_category(age_category):
    if age_category == '만18-34세':
        return (18 + 34) / 2
    elif age_category == '만35-37세':
        return (35 + 37) / 2
    elif age_category == '만38-39세':
        return (38 + 39) / 2
    elif age_category == '만40-42세':
        return (40 + 42) / 2
    elif age_category == '만43-44세':
        return (43 + 44) / 2
    elif age_category == '만45-50세':
        return (45 + 50) / 2
    else:
        return None

# '시술_당시_나이' 변수의 값을 중앙값으로 변환
train['시술_당시_나이'] = train['시술_당시_나이'].apply(convert_age_category)
test['시술_당시_나이'] = test['시술_당시_나이'].apply(convert_age_category)

In [170]:
value_counts_ratio(train, '시술_당시_나이')


시술_당시_나이별 임신_성공_여부 비율 및 갯수

                 0         1      0      1   Total
variable                                          
26.0      0.674737  0.325263  67747  32658  100405
36.0      0.718321  0.281679  40463  15867   56330
38.5      0.779704  0.220296  29780   8414   38194
41.0      0.838043  0.161957  30431   5881   36312
43.5      0.878486  0.121514  10396   1438   11834
47.5      0.825871  0.174129   5497   1159    6656


### 3. 임신 시도 또는 마지막 임신 경과 일수

환자가 처음 임신을 시도한 시점 또는 마지막 임신 이후 현재까지의 경과 연수 (년 단위)

In [171]:
len(train)

249731

In [172]:
value_counts_ratio(train, '임신_시도_또는_마지막_임신_경과_연수')


임신_시도_또는_마지막_임신_경과_연수별 임신_성공_여부 비율 및 갯수

                 0         1    0    1  Total
variable                                     
0.0       0.666667  0.333333    8    4     12
1.0       0.500000  0.500000    2    2      4
2.0       0.809524  0.190476   17    4     21
3.0       0.762500  0.237500   61   19     80
4.0       0.759322  0.240678  224   71    295
5.0       0.753687  0.246313  511  167    678
6.0       0.747126  0.252874  715  242    957
7.0       0.765453  0.234547  904  277   1181
8.0       0.782379  0.217621  888  247   1135
9.0       0.790744  0.209256  786  208    994
10.0      0.778384  0.221616  713  203    916
11.0      0.808036  0.191964  543  129    672
12.0      0.776639  0.223361  379  109    488
13.0      0.835853  0.164147  387   76    463
14.0      0.775701  0.224299  249   72    321
15.0      0.791165  0.208835  197   52    249
16.0      0.845771  0.154229  170   31    201
17.0      0.820896  0.179104  110   24    134
18.0      0.836538  0.163462   87   17

In [173]:
print("결측값 개수:", train['임신_시도_또는_마지막_임신_경과_연수'].isnull().sum())

train['임신_시도_또는_마지막_임신_경과_연수'].fillna(-1, inplace=True)
test['임신_시도_또는_마지막_임신_경과_연수'].fillna(-1, inplace=True)

print("결측값 개수:", train['임신_시도_또는_마지막_임신_경과_연수'].isnull().sum())

결측값 개수: 240664
결측값 개수: 0


In [174]:
value_counts_ratio(train, '임신_시도_또는_마지막_임신_경과_연수')


임신_시도_또는_마지막_임신_경과_연수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-1.0      0.736408  0.263592  177227  63437  240664
 0.0      0.666667  0.333333       8      4      12
 1.0      0.500000  0.500000       2      2       4
 2.0      0.809524  0.190476      17      4      21
 3.0      0.762500  0.237500      61     19      80
 4.0      0.759322  0.240678     224     71     295
 5.0      0.753687  0.246313     511    167     678
 6.0      0.747126  0.252874     715    242     957
 7.0      0.765453  0.234547     904    277    1181
 8.0      0.782379  0.217621     888    247    1135
 9.0      0.790744  0.209256     786    208     994
 10.0     0.778384  0.221616     713    203     916
 11.0     0.808036  0.191964     543    129     672
 12.0     0.776639  0.223361     379    109     488
 13.0     0.835853  0.164147     387     76     463
 14.0     0.775701  0.224299     249     72     321
 15.0     0.791165  0.

### 5. 특정_시술_유형

IVF – 체외 수정 (시험관 아기 시술)  
ICSI – 세포질 내 정자 주입  
IUI – 인공 수정 (자궁 내 주입)  
BLASTOCYST – 배반포 단계에 이식  
AH – 배아를 hatching 시켜주는 과정  
Unknown – 시술 유형 정보가 제공되지 않았거나 알 수 없음  

(*) 특수 기호의 의미  
"/" – 복합적인 시술 방법을 의미 (예: ICSI / BLASTOCYST)  
":" – 동일 시술 내 세부 조합을 의미 (예: ICSI:IVF)  

In [175]:
value_counts_ratio(train, '특정_시술_유형')


특정_시술_유형별 임신_성공_여부 비율 및 갯수

                                            0         1        0        1  \
variable                                                                    
FER                                  0.666667  0.333333      2.0      1.0   
GIFT                                 1.000000  0.000000      1.0      0.0   
ICSI                                 0.727151  0.272849  88972.0  33385.0   
ICSI / AH                            0.782835  0.217165    602.0    167.0   
ICSI / AH:Unknown                    0.500000  0.500000      1.0      1.0   
ICSI / BLASTOCYST                    0.643257  0.356743   1035.0    574.0   
ICSI / BLASTOCYST :ICSI              0.000000  1.000000      0.0      1.0   
ICSI / BLASTOCYST :IVF / BLASTOCYST  0.750000  0.250000      3.0      1.0   
ICSI / BLASTOCYST:IVF / BLASTOCYST   0.666667  0.333333      4.0      2.0   
ICSI:ICSI                            0.990035  0.009965   2285.0     23.0   
ICSI:IVF                             0.765178  

In [176]:
test['특정_시술_유형'].value_counts()

특정_시술_유형
ICSI                                  43229
IVF                                   32247
Unknown                                9213
ICSI:ICSI                               796
ICSI / BLASTOCYST                       584
IVF:IVF                                 405
IVF / BLASTOCYST                        389
ICSI:IVF                                298
ICSI / AH                               264
IVF:ICSI                                133
IVF / AH                                105
ICSI:Unknown                             68
IVF:Unknown                              41
ICSI / BLASTOCYST:IVF / BLASTOCYST        2
IVF:Unknown:Unknown:Unknown               1
Name: count, dtype: int64

갯수 적은 특정 시술 유형 제거

In [ ]:
# '특정_시술_유형' 변수의 값이 'FER' 또는 'GIFT'인 행 제거
train = train.loc[~train['특정_시술_유형'].isin(['FER', 'GIFT'])]
# test = test.loc[~test['특정_시술_유형'].isin(['FER', 'GIFT'])]

In [178]:
# 시술 유형 변환 함수
def transform_treatment(treatment):
    if isinstance(treatment, str):  # treatment가 문자열인지 확인
        if treatment.startswith("ICSI"):
            if "AH" in treatment:
                return "ICSI / AH"
            elif "BLASTOCYST" in treatment:
                return "ICSI / BLASTOCYST"
            return "ICSI"
        elif treatment.startswith("IVF"):
            if "AH" in treatment:
                return "IVF / AH"
            elif "BLASTOCYST" in treatment:
                return "IVF / BLASTOCYST"
            return "IVF"
    return treatment  # 기타 시술은 그대로 반환

# 변환된 열 추가
train['특정_시술_유형'] = train['특정_시술_유형'].apply(transform_treatment)
test['특정_시술_유형'] = test['특정_시술_유형'].apply(transform_treatment)

In [179]:
value_counts_ratio(train, '특정_시술_유형')


특정_시술_유형별 임신_성공_여부 비율 및 갯수

                          0         1      0      1   Total
variable                                                   
ICSI               0.732522  0.267478  92111  33634  125745
ICSI / AH          0.782101  0.217899    603    168     771
ICSI / BLASTOCYST  0.643210  0.356790   1042    578    1620
IVF                0.740766  0.259234  68952  24130   93082
IVF / AH           0.816199  0.183801    262     59     321
IVF / BLASTOCYST   0.633814  0.366186    791    457    1248
Unknown            0.762789  0.237211  20548   6390   26938


In [180]:
test['특정_시술_유형'].value_counts()

특정_시술_유형
ICSI                 44391
IVF                  32827
Unknown               9213
ICSI / BLASTOCYST      586
IVF / BLASTOCYST       389
ICSI / AH              264
IVF / AH               105
Name: count, dtype: int64

In [181]:
# 더미 변수 생성 함수 정의
def create_dummy(x, keyword):
    if isinstance(x, str) and keyword in x:
        return 1
    else:
        return 0

# 더미 변수 생성
train['특정_시술_유형_ICSI_더미'] = train['특정_시술_유형'].apply(lambda x: create_dummy(x, 'ICSI'))
train['특정_시술_유형_AH_더미'] = train['특정_시술_유형'].apply(lambda x: create_dummy(x, 'AH'))
train['특정_시술_유형_BLASTOCYST_더미'] = train['특정_시술_유형'].apply(lambda x: create_dummy(x, 'BLASTOCYST'))

test['특정_시술_유형_ICSI_더미'] = test['특정_시술_유형'].apply(lambda x: create_dummy(x, 'ICSI'))
test['특정_시술_유형_AH_더미'] = test['특정_시술_유형'].apply(lambda x: create_dummy(x, 'AH'))
test['특정_시술_유형_BLASTOCYST_더미'] = test['특정_시술_유형'].apply(lambda x: create_dummy(x, 'BLASTOCYST'))

In [182]:
summarize_group(train, ['특정_시술_유형',
                        '특정_시술_유형_AH_더미',
                        '특정_시술_유형_BLASTOCYST_더미'])

Grouped by: 특정_시술_유형, 특정_시술_유형_AH_더미, 특정_시술_유형_BLASTOCYST_더미

                       group  '임신_성공_여부' count     ratio   Total
0               (ICSI, 0, 0)             33634  0.267478  125745
1          (ICSI / AH, 1, 0)               168  0.217899     771
2  (ICSI / BLASTOCYST, 0, 1)               578  0.356790    1620
3                (IVF, 0, 0)             24130  0.259234   93082
4           (IVF / AH, 1, 0)                59  0.183801     321
5   (IVF / BLASTOCYST, 0, 1)               457  0.366186    1248
6            (Unknown, 0, 0)              6390  0.237211   26938


### 6. 배란_자극_여부

배란 자극 치료가 사용되었는지 여부

In [183]:
value_counts_ratio(train, '배란_자극_여부')


배란_자극_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.762168  0.237832   42391  13228   55619
1         0.731139  0.268861  141920  52188  194108


### 7. 배란_유도_유형

사용된 배란 유도 방법 유형

In [184]:
value_counts_ratio(train, '배란_유도_유형')


배란_유도_유형별 임신_성공_여부 비율 및 갯수

                     0         1         0        1   Total
variable                                                   
기록되지 않은 시행    0.731142  0.268858  141919.0  52187.0  194106
생식선 자극 호르몬    0.000000  1.000000       0.0      1.0       1
세트로타이드 (억제제)  1.000000  0.000000       1.0      0.0       1
알 수 없음        0.762168  0.237832   42391.0  13228.0   55619


In [185]:
test['배란_유도_유형'].value_counts()

배란_유도_유형
기록되지 않은 시행    68494
알 수 없음        19281
Name: count, dtype: int64

In [186]:
# 갯수 적은 배란 유도 유형 제거
train = train.loc[~train['배란_유도_유형'].isin(['생식선 자극 호르몬', '세트로타이드 (억제제)'])]
test = test.loc[~test['배란_유도_유형'].isin(['생식선 자극 호르몬', '세트로타이드 (억제제)'])]

In [187]:
value_counts_ratio(train, '배란_유도_유형')


배란_유도_유형별 임신_성공_여부 비율 및 갯수

                   0         1       0      1   Total
variable                                             
기록되지 않은 시행  0.731142  0.268858  141919  52187  194106
알 수 없음      0.762168  0.237832   42391  13228   55619


In [188]:
train= train.drop(columns=['배란_유도_유형'])
test = test.drop(columns=['배란_유도_유형'])

### 8. 단일_배아_이식_여부

선택적으로 단일 배아를 이식했는지 여부

In [189]:
value_counts_ratio(train, '단일_배아_이식_여부')


단일_배아_이식_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.770177  0.229823  147368  43975  191343
1.0       0.632764  0.367236   36942  21440   58382


### 9. 착상_전_유전_검사_사용_여부

PGS(착상 전 유전 검사)가 시행되었는지 여부

In [190]:
value_counts_ratio(train, '착상_전_유전_검사_사용_여부')


착상_전_유전_검사_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
1.0       0.851729  0.148271  2315  403   2718


In [191]:
# 결측값 개수 확인
print("결측값 개수:", train['착상_전_유전_검사_사용_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['착상_전_유전_검사_사용_여부'].fillna(0, inplace=True)
test['착상_전_유전_검사_사용_여부'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['착상_전_유전_검사_사용_여부'].isnull().sum())

결측값 개수: 247007
결측값 개수: 0


### 10. 착상_전_유전_진단_사용_여부

PGD(착상 전 유전 진단)가 시행되었는지 여부

In [192]:
value_counts_ratio(train, '착상_전_유전_진단_사용_여부')


착상_전_유전_진단_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.737147  0.262853  181728  64801  246529
1.0       0.807885  0.192115    2582    614    3196


In [193]:
summarize_group(train, [
    '착상_전_유전_진단_사용_여부',
     '착상_전_유전_검사_사용_여부'
     ])

Grouped by: 착상_전_유전_진단_사용_여부, 착상_전_유전_검사_사용_여부

        group  '임신_성공_여부' count     ratio   Total
0  (0.0, 0.0)             64400  0.264111  243837
1  (0.0, 1.0)               401  0.148960    2692
2  (1.0, 0.0)               612  0.193060    3170
3  (1.0, 1.0)                 2  0.076923      26


### 11. 남성_주_불임_원인

남성의 주요 불임 원인 여부

In [194]:
value_counts_ratio(train, '남성_주_불임_원인')


남성_주_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.736864  0.263136  178839  63864  242703
1         0.779123  0.220877    5471   1551    7022


### 12. 남성_부_불임_원인

남성의 부차적 불임 원인 여부

In [195]:
value_counts_ratio(train, '남성_부_불임_원인')


남성_부_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.737240  0.262760  181720  64767  246487
1         0.799876  0.200124    2590    648    3238


In [196]:
summarize_group(train, ['남성_주_불임_원인', '남성_부_불임_원인'])

Grouped by: 남성_주_불임_원인, 남성_부_불임_원인

    group  '임신_성공_여부' count     ratio   Total
0  (0, 0)             63216  0.263985  239468
1  (0, 1)               648  0.200309    3235
2  (1, 0)              1551  0.220972    7019
3  (1, 1)                 0  0.000000       3


### 13. 여성_주_불임_원인

여성의 주요 불임 원인 여부

In [197]:
value_counts_ratio(train, '여성_주_불임_원인')


여성_주_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.736813  0.263187  178498  63759  242257
1         0.778254  0.221746    5812   1656    7468


### 14. 여성_부_불임_원인

여성의 부차적 불임 원인 여부

In [198]:
value_counts_ratio(train, '여성_부_불임_원인')


여성_부_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.737258  0.262742  181858  64810  246668
1         0.802094  0.197906    2452    605    3057


In [199]:
summarize_group(train, ['여성_주_불임_원인', '여성_부_불임_원인'])

Grouped by: 여성_주_불임_원인, 여성_부_불임_원인

    group  '임신_성공_여부' count     ratio   Total
0  (0, 0)             63156  0.264022  239207
1  (0, 1)               603  0.197705    3050
2  (1, 0)              1654  0.221686    7461
3  (1, 1)                 2  0.285714       7


### 15. 부부_주_불임_원인

부부의 주요 불임 원인 여부

In [200]:
value_counts_ratio(train, '부부_주_불임_원인')


부부_주_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.736695  0.263305  177998  63619  241617
1         0.778490  0.221510    6312   1796    8108


### 16. 부부_부_불임_원인

부부의 부차적 불임 원인 여부

In [201]:
value_counts_ratio(train, '부부_부_불임_원인')


부부_부_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.737404  0.262596  182540  65004  247544
1         0.811554  0.188446    1770    411    2181


In [202]:
summarize_group(train, ['부부_주_불임_원인', '부부_부_불임_원인'])

Grouped by: 부부_주_불임_원인, 부부_부_불임_원인

    group  '임신_성공_여부' count     ratio   Total
0  (0, 0)             63208  0.263982  239441
1  (0, 1)               411  0.188879    2176
2  (1, 0)              1796  0.221646    8103
3  (1, 1)                 0  0.000000       5


In [203]:
# 두 변수에 대해 그룹별 갯수 카운팅
test.groupby(['부부_주_불임_원인', '부부_부_불임_원인']).size().reset_index(name='count')

,부부_주_불임_원인,부부_부_불임_원인,count
0,0,0,84138
1,0,1,751
2,1,0,2883
3,1,1,3


### 17. 불명확_불임_원인

불명의 원인으로 인한 불임 여부

In [204]:
value_counts_ratio(train, '불명확_불임_원인')


불명확_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.737626  0.262374  137146  48783  185929
1         0.739294  0.260706   47164  16632   63796


### 18. 불임_원인_-_난관_질환

난관 이상으로 인한 불임 여부

In [205]:
value_counts_ratio(train, '불임_원인_-_난관_질환')


불임_원인_-_난관_질환별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.737422  0.262578  157989  56256  214245
1         0.741855  0.258145   26321   9159   35480


### 19. 불임_원인_-_남성_요인

남성 요인으로 인한 불임 여부

In [206]:
value_counts_ratio(train, '불임_원인_-_남성_요인')


불임_원인_-_남성_요인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.750845  0.249155  117990  39153  157143
1         0.716338  0.283662   66320  26262   92582


### 20. 불임_원인_-_배란_장애

배란 장애로 인한 불임 여부

In [207]:
value_counts_ratio(train, '불임_원인_-_배란_장애')


불임_원인_-_배란_장애별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.742091  0.257909  160797  55884  216681
1         0.711566  0.288434   23513   9531   33044


### 22. 불임_원인_-_자궁경부_문제

자궁경부 이상으로 인한 불임 여부

In [208]:
value_counts_ratio(train, '불임_원인_-_자궁경부_문제')


불임_원인_-_자궁경부_문제별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0         0.738041  0.261959  184300.0  65415.0  249715
1         1.000000  0.000000      10.0      0.0      10


In [209]:
# 행 제거
train = train[train['불임_원인_-_자궁경부_문제'] != 1]
test = test[test['불임_원인_-_자궁경부_문제'] != 1]

In [210]:
train = train.drop(columns=['불임_원인_-_자궁경부_문제'])
test = test.drop(columns=['불임_원인_-_자궁경부_문제'])

### 23. 불임_원인_-_자궁내막증

자궁내막증으로 인한 불임 여부

In [211]:
value_counts_ratio(train, '불임_원인_-_자궁내막증')


불임_원인_-_자궁내막증별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.737609  0.262391  170814  60764  231578
1         0.743563  0.256437   13486   4651   18137


### 24. 불임_원인_-_정자_농도

파트너의 정자 농도 문제 여부

In [212]:
value_counts_ratio(train, '불임_원인_-_정자_농도')


불임_원인_-_정자_농도별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.738037  0.261963  184124  65354  249478
1         0.742616  0.257384     176     61     237


### 25. 불임_원인_-_정자_면역학적_요인

파트너의 정자 면역 관련 문제 여부

In [213]:
value_counts_ratio(train, '불임_원인_-_정자_면역학적_요인')


불임_원인_-_정자_면역학적_요인별 임신_성공_여부 비율 및 갯수

                0        1         0        1   Total
variable                                             
0         0.73804  0.26196  184299.0  65415.0  249714
1         1.00000  0.00000       1.0      0.0       1


In [214]:
test['불임_원인_-_정자_면역학적_요인'].value_counts()

불임_원인_-_정자_면역학적_요인
0    87770
1        1
Name: count, dtype: int64

In [215]:
# 행 제거
train = train[train['불임_원인_-_정자_면역학적_요인'] != 1]
test = test[test['불임_원인_-_정자_면역학적_요인'] != 1]

In [216]:
train = train.drop(columns=['불임_원인_-_정자_면역학적_요인'])
test = test.drop(columns=['불임_원인_-_정자_면역학적_요인'])

### 26. 불임_원인_-_정자_운동성

파트너의 정자 운동성 문제 여부

In [217]:
value_counts_ratio(train, '불임_원인_-_정자_운동성')


불임_원인_-_정자_운동성별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.738007  0.261993  184219  65398  249617
1         0.824742  0.175258      80     17      97


### 27. 불임_원인_-_정자_형태

파트너의 정자 형태 문제 여부

In [218]:
value_counts_ratio(train, '불임_원인_-_정자_형태')


불임_원인_-_정자_형태별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.738021  0.261979  184190  65383  249573
1         0.773050  0.226950     109     32     141


### 28. 배아_생성_주요_이유

배아 생성의 주된 목적(시술, 보관 등)

In [219]:
value_counts_ratio(train, '배아_생성_주요_이유')


배아_생성_주요_이유별 임신_성공_여부 비율 및 갯수

                            0         1         0        1   Total
variable                                                          
기증용                  1.000000  0.000000     794.0      0.0     794
기증용, 난자 저장용          1.000000  0.000000      42.0      0.0      42
기증용, 배아 저장용          1.000000  0.000000     112.0      0.0     112
기증용, 배아 저장용, 현재 시술용  0.650000  0.350000      13.0      7.0      20
기증용, 현재 시술용          0.620143  0.379857    2346.0   1437.0    3783
난자 저장용               1.000000  0.000000    1959.0      0.0    1959
난자 저장용, 배아 저장용       1.000000  0.000000       6.0      0.0       6
난자 저장용, 배아 저장용, 연구용  1.000000  0.000000       1.0      0.0       1
난자 저장용, 현재 시술용       0.800000  0.200000       4.0      1.0       5
배아 저장용               0.999130  0.000870    9183.0      8.0    9191
배아 저장용, 현재 시술용       0.734940  0.265060      61.0     22.0      83
연구용, 현재 시술용          1.000000  0.000000       1.0      0.0       1
현재 시술용               0.726421 

In [220]:
# '연구용'이라는 값이 포함된 행 삭제
train = train[~train['배아_생성_주요_이유'].str.contains('연구용', na=False)]
test = test[~test['배아_생성_주요_이유'].str.contains('연구용', na=False)]

In [221]:
value_counts_ratio(train, '배아_생성_주요_이유')


배아_생성_주요_이유별 임신_성공_여부 비율 및 갯수

                            0         1         0        1   Total
variable                                                          
기증용                  1.000000  0.000000     794.0      0.0     794
기증용, 난자 저장용          1.000000  0.000000      42.0      0.0      42
기증용, 배아 저장용          1.000000  0.000000     112.0      0.0     112
기증용, 배아 저장용, 현재 시술용  0.650000  0.350000      13.0      7.0      20
기증용, 현재 시술용          0.620143  0.379857    2346.0   1437.0    3783
난자 저장용               1.000000  0.000000    1959.0      0.0    1959
난자 저장용, 배아 저장용       1.000000  0.000000       6.0      0.0       6
난자 저장용, 현재 시술용       0.800000  0.200000       4.0      1.0       5
배아 저장용               0.999130  0.000870    9183.0      8.0    9191
배아 저장용, 현재 시술용       0.734940  0.265060      61.0     22.0      83
현재 시술용               0.726421  0.273579  169777.0  63940.0  233717


In [222]:
# 제거할 값 목록
values_to_drop = ['기증용', '기증용, 난자 저장용', '기증용, 배아 저장용', '난자 저장용', '난자 저장용, 배아 저장용']

# '배아_생성_주요_이유' 변수의 값이 특정 값인 행 제거
train = train[~train['배아_생성_주요_이유'].isin(values_to_drop)]
test = test[~test['배아_생성_주요_이유'].isin(values_to_drop)]

In [223]:
value_counts_ratio(train, '배아_생성_주요_이유')


배아_생성_주요_이유별 임신_성공_여부 비율 및 갯수

                            0         1       0      1   Total
variable                                                      
기증용, 배아 저장용, 현재 시술용  0.650000  0.350000      13      7      20
기증용, 현재 시술용          0.620143  0.379857    2346   1437    3783
난자 저장용, 현재 시술용       0.800000  0.200000       4      1       5
배아 저장용               0.999130  0.000870    9183      8    9191
배아 저장용, 현재 시술용       0.734940  0.265060      61     22      83
현재 시술용               0.726421  0.273579  169777  63940  233717


In [224]:
# '현재 시술용' 키워드가 포함된 경우 1, 그렇지 않은 경우 0을 부여하는 더미 변수 생성
train['배아_생성_주요_이유_현재시술용'] = train['배아_생성_주요_이유'].str.contains('현재 시술용').astype(int)
test['배아_생성_주요_이유_현재시술용'] = test['배아_생성_주요_이유'].str.contains('현재 시술용').astype(int)

In [225]:
# '기증용' 키워드가 포함된 경우 1, 그렇지 않은 경우 0을 부여하는 더미 변수 생성
train['배아_생성_주요_이유_기증용'] = train['배아_생성_주요_이유'].str.contains('기증용').astype(int)
test['배아_생성_주요_이유_기증용'] = test['배아_생성_주요_이유'].str.contains('기증용').astype(int)

In [226]:
# '난자 저장용' 키워드가 포함된 경우 1, 그렇지 않은 경우 0을 부여하는 더미 변수 생성
train['배아_생성_주요_이유_난자저장용'] = train['배아_생성_주요_이유'].str.contains('난자 저장용').astype(int)
test['배아_생성_주요_이유_난자저장용'] = test['배아_생성_주요_이유'].str.contains('난자 저장용').astype(int)

In [227]:
# '배아 저장용' 키워드가 포함된 경우 1, 그렇지 않은 경우 0을 부여하는 더미 변수 생성
train['배아_생성_주요_이유_배아저장용'] = train['배아_생성_주요_이유'].str.contains('배아 저장용').astype(int)
test['배아_생성_주요_이유_배아저장용'] = test['배아_생성_주요_이유'].str.contains('배아 저장용').astype(int)

In [228]:
summarize_group(train, [
    # '배아_생성_주요_이유',
    '배아_생성_주요_이유_현재시술용',
    '배아_생성_주요_이유_기증용',
    '배아_생성_주요_이유_난자저장용',
    '배아_생성_주요_이유_배아저장용'
                        ])

Grouped by: 배아_생성_주요_이유_현재시술용, 배아_생성_주요_이유_기증용, 배아_생성_주요_이유_난자저장용, 배아_생성_주요_이유_배아저장용

          group  '임신_성공_여부' count     ratio   Total
0  (0, 0, 0, 1)                 8  0.000870    9191
1  (1, 0, 0, 0)             63940  0.273579  233717
2  (1, 0, 0, 1)                22  0.265060      83
3  (1, 0, 1, 0)                 1  0.200000       5
4  (1, 1, 0, 0)              1437  0.379857    3783
5  (1, 1, 0, 1)                 7  0.350000      20


In [229]:
value_counts_ratio(train, '배아_생성_주요_이유')


배아_생성_주요_이유별 임신_성공_여부 비율 및 갯수

                            0         1       0      1   Total
variable                                                      
기증용, 배아 저장용, 현재 시술용  0.650000  0.350000      13      7      20
기증용, 현재 시술용          0.620143  0.379857    2346   1437    3783
난자 저장용, 현재 시술용       0.800000  0.200000       4      1       5
배아 저장용               0.999130  0.000870    9183      8    9191
배아 저장용, 현재 시술용       0.734940  0.265060      61     22      83
현재 시술용               0.726421  0.273579  169777  63940  233717


In [230]:
train['배아_생성_주요_이유'].isnull().sum()

0

In [231]:
value_counts_ratio(train, '배아_생성_주요_이유_난자저장용')


배아_생성_주요_이유_난자저장용별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0         0.734945  0.265055  181380  65414  246794
1         0.800000  0.200000       4      1       5


갯수 적으므로 제거

In [232]:
# 변수 제거
train = train.drop(columns=['배아_생성_주요_이유_난자저장용'])
test = test.drop(columns=['배아_생성_주요_이유_난자저장용'])

In [233]:
train = train.drop(columns=['배아_생성_주요_이유'])
test = test.drop(columns=['배아_생성_주요_이유'])

### 29. 총_시술_횟수

IVF 및 DI 포함한 전체 시술 횟수

In [234]:
value_counts_ratio(train, '총_시술_횟수')


총_시술_횟수별 임신_성공_여부 비율 및 갯수

                 0         1      0      1  Total
variable                                         
0회        0.703322  0.296678  67258  28371  95629
1회        0.744891  0.255109  41112  14080  55192
2회        0.750033  0.249967  28373   9456  37829
3회        0.756496  0.243504  17615   5670  23285
4회        0.764756  0.235244  10780   3316  14096
5회        0.774629  0.225371   6424   1869   8293
6회 이상     0.787335  0.212665   9822   2653  12475


### 30. 클리닉_내_총_시술_횟수

특정 클리닉에서 받은 IVF 및 DI 시술 횟수

In [235]:
value_counts_ratio(train, '클리닉_내_총_시술_횟수')


클리닉_내_총_시술_횟수별 임신_성공_여부 비율 및 갯수

                 0         1      0      1   Total
variable                                          
0회        0.711514  0.288486  84663  34327  118990
1회        0.749274  0.250726  43356  14508   57864
2회        0.755599  0.244401  24965   8075   33040
3회        0.759404  0.240596  13102   4151   17253
4회        0.767789  0.232211   7003   2118    9121
5회        0.780441  0.219559   3679   1035    4714
6회 이상     0.793536  0.206464   4616   1201    5817


### 31. IVF_시술_횟수

이전에 수행된 IVF 시술 횟수

In [236]:
value_counts_ratio(train, 'IVF_시술_횟수')


IVF_시술_횟수별 임신_성공_여부 비율 및 갯수

                 0         1      0      1  Total
variable                                         
0회        0.703490  0.296510  69355  29232  98587
1회        0.744177  0.255823  42171  14497  56668
2회        0.749725  0.250275  28623   9555  38178
3회        0.760272  0.239728  17227   5432  22659
4회        0.767990  0.232010  10192   3079  13271
5회        0.782431  0.217569   5923   1647   7570
6회 이상     0.800020  0.199980   7893   1973   9866


### 32. DI_시술_횟수

이전에 수행된 DI 시술 횟수

In [237]:
value_counts_ratio(train, 'DI_시술_횟수')


DI_시술_횟수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0회        0.735099  0.264901  175061  63085  238146
1회        0.758045  0.241955    1272    406    1678
2회        0.734442  0.265558    1369    495    1864
3회        0.708485  0.291515    1553    639    2192
4회        0.721834  0.278166     724    279    1003
5회        0.735818  0.264182     454    163     617
6회 이상     0.732102  0.267898     951    348    1299


### 33. 총_임신_횟수

IVF 및 DI 포함한 총 임신 횟수

In [238]:
value_counts_ratio(train, '총_임신_횟수')


총_임신_횟수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0회        0.735637  0.264363  146149  52521  198670
1회        0.730868  0.269132   30293  11155   41448
2회        0.738370  0.261630    4349   1541    5890
3회        0.746763  0.253237     519    176     695
4회        0.767442  0.232558      66     20      86
5회        0.857143  0.142857       6      1       7
6회 이상     0.666667  0.333333       2      1       3


### 34. IVF_임신_횟수

IVF 시술로 인한 임신 횟수

In [239]:
value_counts_ratio(train, 'IVF_임신_횟수')


IVF_임신_횟수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0회        0.735542  0.264458  146667.0  52733.0  199400
1회        0.730794  0.269206   29907.0  11017.0   40924
2회        0.741755  0.258245    4251.0   1480.0    5731
3회        0.750000  0.250000     492.0    164.0     656
4회        0.762500  0.237500      61.0     19.0      80
5회        0.857143  0.142857       6.0      1.0       7
6회 이상     0.000000  1.000000       0.0      1.0       1


### 35. DI_임신_횟수

DI 시술로 인한 임신 횟수

In [240]:
value_counts_ratio(train, 'DI_임신_횟수')


DI_임신_횟수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0회        0.735069  0.264931  180780.0  65156.0  245936
1회        0.705333  0.294667     529.0    221.0     750
2회        0.646465  0.353535      64.0     35.0      99
3회        0.727273  0.272727       8.0      3.0      11
4회        1.000000  0.000000       1.0      0.0       1
5회        1.000000  0.000000       2.0      0.0       2


### 36. 총_출산_횟수

IVF 및 DI 시술로 출산한 횟수

In [241]:
value_counts_ratio(train, '총_출산_횟수')


총_출산_횟수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0회        0.736236  0.263764  155468.0  55698.0  211166
1회        0.727286  0.272714   24295.0   9110.0   33405
2회        0.728814  0.271186    1505.0    560.0    2065
3회        0.700680  0.299320     103.0     44.0     147
4회        0.769231  0.230769      10.0      3.0      13
5회        1.000000  0.000000       1.0      0.0       1
6회 이상     1.000000  0.000000       2.0      0.0       2


### 37. IVF_출산_횟수

IVF 시술로 출산한 횟수

In [242]:
value_counts_ratio(train, 'IVF_출산_횟수')


IVF_출산_횟수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0회        0.736134  0.263866  155869.0  55871.0  211740
1회        0.727483  0.272517   23948.0   8971.0   32919
2회        0.734171  0.265829    1461.0    529.0    1990
3회        0.698529  0.301471      95.0     41.0     136
4회        0.769231  0.230769      10.0      3.0      13
5회        1.000000  0.000000       1.0      0.0       1


### 38. DI_출산_횟수

DI 시술로 출산한 횟수

In [243]:
value_counts_ratio(train, 'DI_출산_횟수')


DI_출산_횟수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0회        0.735055  0.264945  180953.0  65223.0  246176
1회        0.694590  0.305410     398.0    175.0     573
2회        0.652174  0.347826      30.0     16.0      46
3회        0.500000  0.500000       1.0      1.0       2
5회        1.000000  0.000000       2.0      0.0       2


In [244]:
# 숫자 매핑
mapping = {
    '0회': 0, '1회': 1, '2회': 2, '3회': 3, 
    '4회': 4, '5회': 5, '6회 이상': 6
}

# 변환할 열 목록
columns_to_map = [
    '총_시술_횟수', '클리닉_내_총_시술_횟수', 'IVF_시술_횟수', 
    'DI_시술_횟수', '총_임신_횟수', 'IVF_임신_횟수', 'DI_임신_횟수', 
    '총_출산_횟수', 'IVF_출산_횟수', 'DI_출산_횟수'
]

# 숫자로 변환
for col in columns_to_map:
    train[col] = train[col].map(mapping)
    test[col] = test[col].map(mapping)

In [245]:
# 비율 계산 (train 데이터프레임)
train['IVF_임신_시술_비율'] = np.where(train['IVF_시술_횟수'] > 0, train['IVF_임신_횟수'] / train['IVF_시술_횟수'], 0)
train['DI_임신_시술_비율'] = np.where(train['DI_시술_횟수'] > 0, train['DI_임신_횟수'] / train['DI_시술_횟수'], 0)
train['임신_성공률'] = np.where(train['총_시술_횟수'] > 0, train['총_임신_횟수'] / train['총_시술_횟수'], 0)

# 비율 계산 (test 데이터프레임)
test['IVF_임신_시술_비율'] = np.where(test['IVF_시술_횟수'] > 0, test['IVF_임신_횟수'] / test['IVF_시술_횟수'], 0)
test['DI_임신_시술_비율'] = np.where(test['DI_시술_횟수'] > 0, test['DI_임신_횟수'] / test['DI_시술_횟수'], 0)
test['임신_성공률'] = np.where(test['총_시술_횟수'] > 0, test['총_임신_횟수'] / test['총_시술_횟수'], 0)

In [246]:
# 변수 리스트
columns = [
    # '클리닉_내_총_시술_횟수', 
    '총_시술_횟수', '총_출산_횟수', '총_임신_횟수',
    # 'IVF_시술_횟수', 'IVF_임신_횟수', 'IVF_출산_횟수', 'IVF_임신_시술_비율',
    'DI_시술_횟수', 'DI_임신_횟수', 'DI_출산_횟수',
    'DI_임신_시술_비율', '임신_성공률'
]

# 변수 제거
train = train.drop(columns=columns)
test = test.drop(columns=columns)

### 39. 총_생성_배아_수

해당 시술에서 생성된 배아의 총 개수

In [247]:
value_counts_ratio(train, '총_생성_배아_수')


총_생성_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1        0       1  Total
variable                                            
0.0       0.817857  0.182143  41166.0  9168.0  50334
1.0       0.904962  0.095038  13750.0  1444.0  15194
2.0       0.819671  0.180329  15709.0  3456.0  19165
3.0       0.757942  0.242058  15866.0  5067.0  20933
4.0       0.722302  0.277698  15588.0  5993.0  21581
5.0       0.694262  0.305738  14156.0  6234.0  20390
6.0       0.676561  0.323439  12582.0  6015.0  18597
7.0       0.661190  0.338810  10583.0  5423.0  16006
8.0       0.649666  0.350334   8840.0  4767.0  13607
9.0       0.645228  0.354772   7173.0  3944.0  11117
10.0      0.639165  0.360835   5787.0  3267.0   9054
11.0      0.632731  0.367269   4481.0  2601.0   7082
12.0      0.646935  0.353065   3694.0  2016.0   5710
13.0      0.633712  0.366288   2737.0  1582.0   4319
14.0      0.643730  0.356270   2161.0  1196.0   3357
15.0      0.642147  0.357853   1615.0   900.0   2515
16.0      0.6527

### 40. 미세주입된_난자_수

정자 미세주입(ICSI)으로 처리된 난자의 수

In [248]:
value_counts_ratio(train, '미세주입된_난자_수')


미세주입된_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1        0        1   Total
variable                                              
0.0       0.743005  0.256995  92548.0  32011.0  124559
1.0       0.942491  0.057509   4261.0    260.0    4521
2.0       0.888624  0.111376   5585.0    700.0    6285
3.0       0.833270  0.166730   6587.0   1318.0    7905
4.0       0.789578  0.210422   7591.0   2023.0    9614
5.0       0.744320  0.255680   7895.0   2712.0   10607
6.0       0.722916  0.277084   7994.0   3064.0   11058
7.0       0.704491  0.295509   7326.0   3073.0   10399
8.0       0.693395  0.306605   6719.0   2971.0    9690
9.0       0.681363  0.318637   5857.0   2739.0    8596
10.0      0.665116  0.334884   5148.0   2592.0    7740
11.0      0.649946  0.350054   4237.0   2282.0    6519
12.0      0.660075  0.339925   3705.0   1908.0    5613
13.0      0.641781  0.358219   2983.0   1665.0    4648
14.0      0.661286  0.338714   2540.0   1301.0    3841
15.0      0.650345  0.349655   207

### 41. 미세주입에서_생성된_배아_수

ICSI를 통해 생성된 배아의 수

In [249]:
value_counts_ratio(train, '미세주입에서_생성된_배아_수')


미세주입에서_생성된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1        0        1   Total
variable                                              
0.0       0.751026  0.248974  96627.0  32033.0  128660
1.0       0.908328  0.091672   8987.0    907.0    9894
2.0       0.817692  0.182308  10316.0   2300.0   12616
3.0       0.758101  0.241899  10458.0   3337.0   13795
4.0       0.719365  0.280635   9974.0   3891.0   13865
5.0       0.689305  0.310695   8901.0   4012.0   12913
6.0       0.671477  0.328523   7771.0   3802.0   11573
7.0       0.658950  0.341050   6262.0   3241.0    9503
8.0       0.650816  0.349184   5064.0   2717.0    7781
9.0       0.641142  0.358858   3952.0   2212.0    6164
10.0      0.635514  0.364486   3060.0   1755.0    4815
11.0      0.623829  0.376171   2330.0   1405.0    3735
12.0      0.662185  0.337815   1970.0   1005.0    2975
13.0      0.641860  0.358140   1380.0    770.0    2150
14.0      0.645907  0.354093   1089.0    597.0    1686
15.0      0.653687  0.346313 

### 42. 이식된_배아_수

이식된 배아의 총 수

In [250]:
value_counts_ratio(train, '이식된_배아_수')


이식된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1      0      1   Total
variable                                          
0.0       0.999099  0.000901  33267     30   33297
1.0       0.686442  0.313558  64378  29407   93785
2.0       0.688895  0.311105  76355  34482  110837
3.0       0.831532  0.168468   7384   1496    8880


### 43. 미세주입_배아_이식_수

ICSI 배아 중 이식된 배아의 수

In [251]:
value_counts_ratio(train, '미세주입_배아_이식_수')


미세주입_배아_이식_수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.770763  0.229237  104453  31066  135519
1.0       0.680546  0.319454   31627  14846   46473
2.0       0.687673  0.312327   41041  18640   59681
3.0       0.831643  0.168357    4263    863    5126


### 44. 저장된_배아_수

환자를 위해 저장된 배아의 수

In [252]:
value_counts_ratio(train, '저장된_배아_수')


저장된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.773641  0.226359  126676.0  37064.0  163740
1.0       0.654330  0.345670   13313.0   7033.0   20346
2.0       0.629210  0.370790   12498.0   7365.0   19863
3.0       0.629701  0.370299    8489.0   4992.0   13481
4.0       0.639003  0.360997    5898.0   3332.0    9230
5.0       0.649614  0.350386    3873.0   2089.0    5962
6.0       0.666423  0.333577    2733.0   1368.0    4101
7.0       0.704124  0.295876    1861.0    782.0    2643
8.0       0.713597  0.286403    1333.0    535.0    1868
9.0       0.740078  0.259922     951.0    334.0    1285
10.0      0.780415  0.219585     789.0    222.0    1011
11.0      0.828076  0.171924     525.0    109.0     634
12.0      0.840824  0.159176     449.0     85.0     534
13.0      0.911616  0.088384     361.0     35.0     396
14.0      0.936232  0.063768     323.0     22.0     345
15.0      0.928854 

In [253]:
# '저장된_배아_수' 변수의 값이 25 이상인 행 제거
train = train[train['저장된_배아_수'] < 25]
test = test[test['저장된_배아_수'] < 25]

In [254]:
value_counts_ratio(train, '저장된_배아_수')


저장된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.773641  0.226359  126676.0  37064.0  163740
1.0       0.654330  0.345670   13313.0   7033.0   20346
2.0       0.629210  0.370790   12498.0   7365.0   19863
3.0       0.629701  0.370299    8489.0   4992.0   13481
4.0       0.639003  0.360997    5898.0   3332.0    9230
5.0       0.649614  0.350386    3873.0   2089.0    5962
6.0       0.666423  0.333577    2733.0   1368.0    4101
7.0       0.704124  0.295876    1861.0    782.0    2643
8.0       0.713597  0.286403    1333.0    535.0    1868
9.0       0.740078  0.259922     951.0    334.0    1285
10.0      0.780415  0.219585     789.0    222.0    1011
11.0      0.828076  0.171924     525.0    109.0     634
12.0      0.840824  0.159176     449.0     85.0     534
13.0      0.911616  0.088384     361.0     35.0     396
14.0      0.936232  0.063768     323.0     22.0     345
15.0      0.928854 

### 45. 미세주입_후_저장된_배아_수

ICSI 처리된 배아 중 저장된 배아의 수

In [255]:
value_counts_ratio(train, '미세주입_후_저장된_배아_수')


미세주입_후_저장된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.751965  0.248035  151178.0  49866.0  201044
1.0       0.653175  0.346825    8053.0   4276.0   12329
2.0       0.624559  0.375441    7085.0   4259.0   11344
3.0       0.633005  0.366995    4695.0   2722.0    7417
4.0       0.645075  0.354925    3137.0   1726.0    4863
5.0       0.655275  0.344725    1975.0   1039.0    3014
6.0       0.672191  0.327809    1298.0    633.0    1931
7.0       0.741176  0.258824     945.0    330.0    1275
8.0       0.740528  0.259472     645.0    226.0     871
9.0       0.767480  0.232520     472.0    143.0     615
10.0      0.830472  0.169528     387.0     79.0     466
11.0      0.851724  0.148276     247.0     43.0     290
12.0      0.852713  0.147287     220.0     38.0     258
13.0      0.925000  0.075000     185.0     15.0     200
14.0      0.971429  0.028571     170.0      5.0     175
15.0      0.

### 46. 해동된_배아_수

동결 해동된 배아의 수

In [256]:
value_counts_ratio(train, '해동된_배아_수')


해동된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.727675  0.272325  150139.0  56188.0  206327
1.0       0.784707  0.215293   10242.0   2810.0   13052
2.0       0.761183  0.238817    9189.0   2883.0   12072
3.0       0.794805  0.205195    4284.0   1106.0    5390
4.0       0.778487  0.221513    2685.0    764.0    3449
5.0       0.766012  0.233988    1483.0    453.0    1936
6.0       0.752336  0.247664    1127.0    371.0    1498
7.0       0.759463  0.240537     622.0    197.0     819
8.0       0.698264  0.301736     523.0    226.0     749
9.0       0.672330  0.327670     277.0    135.0     412
10.0      0.741279  0.258721     255.0     89.0     344
11.0      0.756410  0.243590     118.0     38.0     156
12.0      0.673611  0.326389      97.0     47.0     144
13.0      0.679012  0.320988      55.0     26.0      81
14.0      0.720000  0.280000      54.0     21.0      75
15.0      0.714286 

### 47. 해동_난자_수

동결 해동된 난자의 수

In [257]:
value_counts_ratio(train, '해동_난자_수')


해동_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.734583  0.265417  180158.0  65094.0  245252
1.0       0.923077  0.076923      24.0      2.0      26
2.0       0.907692  0.092308      59.0      6.0      65
3.0       0.907692  0.092308      59.0      6.0      65
4.0       0.892308  0.107692     116.0     14.0     130
5.0       0.859813  0.140187      92.0     15.0     107
6.0       0.800000  0.200000     136.0     34.0     170
7.0       0.711268  0.288732     101.0     41.0     142
8.0       0.703196  0.296804     154.0     65.0     219
9.0       0.780952  0.219048      82.0     23.0     105
10.0      0.766667  0.233333      69.0     21.0      90
11.0      0.761194  0.238806      51.0     16.0      67
12.0      0.657143  0.342857      46.0     24.0      70
13.0      0.812500  0.187500      26.0      6.0      32
14.0      0.613636  0.386364      27.0     17.0      44
15.0      0.594595  

In [258]:
# '해동_난자_수' 변수의 값이 25 이상인 행 제거
train = train[train['해동_난자_수'] < 25]
test = test[test['해동_난자_수'] < 25]

In [259]:
value_counts_ratio(train, '해동_난자_수')


해동_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.734583  0.265417  180158.0  65094.0  245252
1.0       0.923077  0.076923      24.0      2.0      26
2.0       0.907692  0.092308      59.0      6.0      65
3.0       0.907692  0.092308      59.0      6.0      65
4.0       0.892308  0.107692     116.0     14.0     130
5.0       0.859813  0.140187      92.0     15.0     107
6.0       0.800000  0.200000     136.0     34.0     170
7.0       0.711268  0.288732     101.0     41.0     142
8.0       0.703196  0.296804     154.0     65.0     219
9.0       0.780952  0.219048      82.0     23.0     105
10.0      0.766667  0.233333      69.0     21.0      90
11.0      0.761194  0.238806      51.0     16.0      67
12.0      0.657143  0.342857      46.0     24.0      70
13.0      0.812500  0.187500      26.0      6.0      32
14.0      0.613636  0.386364      27.0     17.0      44
15.0      0.594595  

### 48. 수집된_신선_난자_수

신선 난자가 수집된 수

In [260]:
value_counts_ratio(train, '수집된_신선_난자_수')


수집된_신선_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1        0        1  Total
variable                                             
0.0       0.759120  0.240880  40805.0  12948.0  53753
1.0       0.947136  0.052864   4497.0    251.0   4748
2.0       0.907176  0.092824   5727.0    586.0   6313
3.0       0.864807  0.135193   6915.0   1081.0   7996
4.0       0.821356  0.178644   8092.0   1760.0   9852
5.0       0.787537  0.212463   8948.0   2414.0  11362
6.0       0.770671  0.229329   9591.0   2854.0  12445
7.0       0.743616  0.256384   9609.0   3313.0  12922
8.0       0.721284  0.278716   9482.0   3664.0  13146
9.0       0.714722  0.285278   9137.0   3647.0  12784
10.0      0.693586  0.306414   8726.0   3855.0  12581
11.0      0.678837  0.321163   8032.0   3800.0  11832
12.0      0.667531  0.332469   7206.0   3589.0  10795
13.0      0.660346  0.339654   6443.0   3314.0   9757
14.0      0.663255  0.336745   5897.0   2994.0   8891
15.0      0.658083  0.341917   5060.0   2629.0   7

### 49. 저장된_신선_난자_수

저장된 신선 난자의 수

In [261]:
value_counts_ratio(train, '저장된_신선_난자_수')


저장된_신선_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.734324  0.265676  180687.0  65372.0  246059
1.0       0.826087  0.173913      19.0      4.0      23
2.0       0.923077  0.076923      24.0      2.0      26
3.0       0.933333  0.066667      42.0      3.0      45
4.0       0.923077  0.076923      48.0      4.0      52
5.0       0.914286  0.085714      64.0      6.0      70
6.0       0.924528  0.075472      49.0      4.0      53
7.0       0.904762  0.095238      38.0      4.0      42
8.0       0.975000  0.025000      39.0      1.0      40
9.0       0.916667  0.083333      33.0      3.0      36
10.0      0.976190  0.023810      41.0      1.0      42
11.0      0.888889  0.111111      32.0      4.0      36
12.0      0.913043  0.086957      21.0      2.0      23
13.0      1.000000  0.000000      25.0      0.0      25
14.0      0.923077  0.076923      24.0      2.0      26
15.0      0.9375

In [262]:
# '저장된_신선_난자_수' 변수의 값이 20 이상인 행 제거
train = train[train['저장된_신선_난자_수'] < 20]
test = test[test['저장된_신선_난자_수'] < 20]

In [263]:
value_counts_ratio(train, '저장된_신선_난자_수')


저장된_신선_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.734324  0.265676  180687.0  65372.0  246059
1.0       0.826087  0.173913      19.0      4.0      23
2.0       0.923077  0.076923      24.0      2.0      26
3.0       0.933333  0.066667      42.0      3.0      45
4.0       0.923077  0.076923      48.0      4.0      52
5.0       0.914286  0.085714      64.0      6.0      70
6.0       0.924528  0.075472      49.0      4.0      53
7.0       0.904762  0.095238      38.0      4.0      42
8.0       0.975000  0.025000      39.0      1.0      40
9.0       0.916667  0.083333      33.0      3.0      36
10.0      0.976190  0.023810      41.0      1.0      42
11.0      0.888889  0.111111      32.0      4.0      36
12.0      0.913043  0.086957      21.0      2.0      23
13.0      1.000000  0.000000      25.0      0.0      25
14.0      0.923077  0.076923      24.0      2.0      26
15.0      0.9375

### 50. 혼합된_난자_수

정자와 혼합된 총 난자 수

In [264]:
value_counts_ratio(train, '혼합된_난자_수')


혼합된_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1        0       1  Total
variable                                            
0.0       0.790131  0.209869  34460.0  9153.0  43613
1.0       0.939201  0.060799   6040.0   391.0   6431
2.0       0.892040  0.107960   7957.0   963.0   8920
3.0       0.837464  0.162536   9259.0  1797.0  11056
4.0       0.794334  0.205666  10795.0  2795.0  13590
5.0       0.750129  0.249871  11621.0  3871.0  15492
6.0       0.735604  0.264396  12008.0  4316.0  16324
7.0       0.714366  0.285634  11397.0  4557.0  15954
8.0       0.697610  0.302390  10626.0  4606.0  15232
9.0       0.689239  0.310761   9710.0  4378.0  14088
10.0      0.672185  0.327815   8729.0  4257.0  12986
11.0      0.661051  0.338949   7651.0  3923.0  11574
12.0      0.660670  0.339330   6721.0  3452.0  10173
13.0      0.646262  0.353738   5627.0  3080.0   8707
14.0      0.658853  0.341147   4975.0  2576.0   7551
15.0      0.656462  0.343538   4267.0  2233.0   6500
16.0      0.64760

### 51. 파트너_정자와_혼합된_난자_수

파트너 정자와 혼합된 난자의 수

In [265]:
value_counts_ratio(train, '파트너_정자와_혼합된_난자_수')


파트너_정자와_혼합된_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1        0        1  Total
variable                                             
0.0       0.771288  0.228712  46140.0  13682.0  59822
1.0       0.939187  0.060813   5359.0    347.0   5706
2.0       0.892140  0.107860   7196.0    870.0   8066
3.0       0.837594  0.162406   8453.0   1639.0  10092
4.0       0.796469  0.203531   9881.0   2525.0  12406
5.0       0.750844  0.249156  10674.0   3542.0  14216
6.0       0.737162  0.262838  11053.0   3941.0  14994
7.0       0.716823  0.283177  10533.0   4161.0  14694
8.0       0.698257  0.301743   9814.0   4241.0  14055
9.0       0.690715  0.309285   9009.0   4034.0  13043
10.0      0.674025  0.325975   8091.0   3913.0  12004
11.0      0.660125  0.339875   7064.0   3637.0  10701
12.0      0.660988  0.339012   6247.0   3204.0   9451
13.0      0.646546  0.353454   5195.0   2840.0   8035
14.0      0.659647  0.340353   4636.0   2392.0   7028
15.0      0.656592  0.343408   3954.0   2068.

### 52. 기증자_정자와_혼합된_난자_수

기증자 정자와 혼합된 난자의 수

In [266]:
value_counts_ratio(train, '기증자_정자와_혼합된_난자_수')


기증자_정자와_혼합된_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.735792  0.264208  170115.0  61085.0  231200
1.0       0.937415  0.062585     689.0     46.0     735
2.0       0.887227  0.112773     771.0     98.0     869
3.0       0.835700  0.164300     824.0    162.0     986
4.0       0.770000  0.230000     924.0    276.0    1200
5.0       0.737490  0.262510     958.0    341.0    1299
6.0       0.714916  0.285084     973.0    388.0    1361
7.0       0.689351  0.310649     861.0    388.0    1249
8.0       0.690784  0.309216     802.0    359.0    1161
9.0       0.670319  0.329681     673.0    331.0    1004
10.0      0.646237  0.353763     601.0    329.0     930
11.0      0.667076  0.332924     543.0    271.0     814
12.0      0.651584  0.348416     432.0    231.0     663
13.0      0.634841  0.365159     379.0    218.0     597
14.0      0.643460  0.356540     305.0    169.0     474
15.0      0

### 53. 난자_출처

난자 출처 정보

In [267]:
value_counts_ratio(train, '난자_출처')


난자_출처별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
기증 제공     0.684411  0.315589   10787   4974   15761
본인 제공     0.738233  0.261767  170455  60441  230896


In [268]:
test['난자_출처'].value_counts()

난자_출처
본인 제공    81234
기증 제공     5489
Name: count, dtype: int64

### 54. 정자_출처

정자 출처 정보

In [269]:
value_counts_ratio(train, '정자_출처')


정자_출처별 임신_성공_여부 비율 및 갯수

                    0         1         0        1   Total
variable                                                  
기증 제공        0.720428  0.279572   14879.0   5774.0   20653
미할당          0.893443  0.106557     109.0     13.0     122
배우자 및 기증 제공  1.000000  0.000000      14.0      0.0      14
배우자 제공       0.736005  0.263995  166240.0  59628.0  225868


In [270]:
test['정자_출처'].value_counts()

정자_출처
배우자 제공         79434
기증 제공           7251
미할당               33
배우자 및 기증 제공        5
Name: count, dtype: int64

In [271]:
# '정자_출처' 변수의 값이 '배우자 및 기증 제공'인 행 제거
train = train[train['정자_출처'] != '배우자 및 기증 제공']
test = test[test['정자_출처'] != '배우자 및 기증 제공']

In [272]:
value_counts_ratio(train, '정자_출처')


정자_출처별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
기증 제공     0.720428  0.279572   14879   5774   20653
미할당       0.893443  0.106557     109     13     122
배우자 제공    0.736005  0.263995  166240  59628  225868


### 55. 난자_기증자_나이

난자 기증자의 등록 당시 나이

In [273]:
value_counts_ratio(train, '난자_기증자_나이')


난자_기증자_나이별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
만20세 이하   0.738095  0.261905     217     77     294
만21-25세   0.669811  0.330189    1562    770    2332
만26-30세   0.651658  0.348342    3242   1733    4975
만31-35세   0.694641  0.305359    4420   1943    6363
알 수 없음    0.738300  0.261700  171787  60892  232679


### 56. 정자_기증자_나이

정자 기증자의 등록 당시 나이

In [274]:
value_counts_ratio(train, '정자_기증자_나이')


정자_기증자_나이별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
만20세 이하   0.761214  0.238786     577    181     758
만21-25세   0.726486  0.273514    3190   1201    4391
만26-30세   0.713768  0.286232    2758   1106    3864
만31-35세   0.715951  0.284049    2702   1072    3774
만36-40세   0.715892  0.284108    2928   1162    4090
만41-45세   0.722488  0.277512    2114    812    2926
알 수 없음    0.736021  0.263979  166959  59881  226840


### 57. 동결_배아_사용_여부

동결 배아가 사용되었는지 여부

In [275]:
value_counts_ratio(train, '동결_배아_사용_여부')


동결_배아_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.727837  0.272163  150323  56211  206534
1.0       0.770525  0.229475   30905   9204   40109


### 58. 신선_배아_사용_여부

신선 배아가 사용되었는지 여부

In [276]:
value_counts_ratio(train, '신선_배아_사용_여부')


신선_배아_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.770341  0.229659   30742   9165   39907
1.0       0.727914  0.272086  150486  56250  206736


### 59. 기증_배아_사용_여부

기증 배아가 사용되었는지 여부

In [277]:
value_counts_ratio(train, '기증_배아_사용_여부')


기증_배아_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.735406  0.264594  179575  64610  244185
1.0       0.672498  0.327502    1653    805    2458


In [278]:
summarize_group(train, ['신선_배아_사용_여부', '동결_배아_사용_여부', '기증_배아_사용_여부'])

Grouped by: 신선_배아_사용_여부, 동결_배아_사용_여부, 기증_배아_사용_여부

             group  '임신_성공_여부' count     ratio   Total
0  (0.0, 0.0, 0.0)                 1  0.333333       3
1  (0.0, 1.0, 0.0)              8916  0.229103   38917
2  (0.0, 1.0, 1.0)               248  0.251266     987
3  (1.0, 0.0, 0.0)             55653  0.271399  205060
4  (1.0, 0.0, 1.0)               557  0.378654    1471
5  (1.0, 1.0, 0.0)                40  0.195122     205


### 60. 대리모_여부

환자가 대리모로 활동했는지 여부

In [279]:
value_counts_ratio(train, '대리모_여부')


대리모_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.734894  0.265106  180495  65112  245607
1.0       0.707529  0.292471     733    303    1036


### 61. PGD_시술_여부

해당 주기가 PGD 치료로 분류되는지 여부

In [280]:
value_counts_ratio(train, 'PGD_시술_여부')


PGD_시술_여부별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
1.0       0.720514  0.279486  1570  609   2179


In [281]:
train['PGD_시술_여부'].isnull().sum()

244464

In [282]:
# 결측값 개수 확인
print("결측값 개수:", train['PGD_시술_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['PGD_시술_여부'].fillna(0, inplace=True)
test['PGD_시술_여부'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['PGD_시술_여부'].isnull().sum())

결측값 개수: 244464
결측값 개수: 0


In [283]:
value_counts_ratio(train, 'PGD_시술_여부')


PGD_시술_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.734906  0.265094  179658  64806  244464
1.0       0.720514  0.279486    1570    609    2179


### 62. PGS_시술_여부

해당 주기가 PGS 치료로 분류되는지 여부

In [284]:
value_counts_ratio(train, 'PGS_시술_여부')


PGS_시술_여부별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
1.0       0.793568  0.206432  1530  398   1928


In [285]:
# 결측값 개수 확인
print("결측값 개수:", train['PGS_시술_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['PGS_시술_여부'].fillna(0, inplace=True)
test['PGS_시술_여부'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['PGS_시술_여부'].isnull().sum())

결측값 개수: 244715
결측값 개수: 0


In [286]:
summarize_group(train, ['PGD_시술_여부', 'PGS_시술_여부'])

Grouped by: PGD_시술_여부, PGS_시술_여부

        group  '임신_성공_여부' count     ratio   Total
0  (0.0, 0.0)             64408  0.265561  242536
1  (0.0, 1.0)               398  0.206432    1928
2  (1.0, 0.0)               609  0.279486    2179


### 63. 난자_채취_경과일

첫 시점으로부터 난자 채취까지 경과한 일수

In [287]:
value_counts_ratio(train, '난자_채취_경과일')


난자_채취_경과일별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.731623  0.268377  143017  52462  195479


In [288]:
print("결측값 개수:", train['난자_채취_경과일'].isnull().sum())

train['난자_채취_경과일'].fillna(-1, inplace=True)
test['난자_채취_경과일'].fillna(-1, inplace=True)

print("결측값 개수:", train['난자_채취_경과일'].isnull().sum())

결측값 개수: 51164
결측값 개수: 0


### 64. 난자_해동_경과일

첫 시점으로부터 난자 해동까지 경과한 일수

In [289]:
value_counts_ratio(train, '난자_해동_경과일')


난자_해동_경과일별 임신_성공_여부 비율 및 갯수

                 0         1       0      1  Total
variable                                          
0.0       0.775826  0.224174  1104.0  319.0   1423
1.0       1.000000  0.000000     2.0    0.0      2


In [290]:
# '난자_해동_경과일' 변수의 값이 1인 행 제거
train = train[train['난자_해동_경과일'] != 1]
test = test[test['난자_해동_경과일'] != 1]

In [291]:
train['난자_해동_경과일'].isnull().sum()

245218

In [292]:
print("결측값 개수:", train['난자_해동_경과일'].isnull().sum())

train['난자_해동_경과일'].fillna(-1, inplace=True)
test['난자_해동_경과일'].fillna(-1, inplace=True)

print("결측값 개수:", train['난자_해동_경과일'].isnull().sum())

결측값 개수: 245218
결측값 개수: 0


### 65. 난자_혼합_경과일

첫 시점으로부터 난자와 정자 혼합까지 경과한 일수

In [293]:
value_counts_ratio(train, '난자_혼합_경과일')


난자_혼합_경과일별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
0.0       0.722413  0.277587  145598.0  55946.0  201544
1.0       0.845679  0.154321     411.0     75.0     486
2.0       0.752475  0.247525      76.0     25.0     101
3.0       0.720588  0.279412      49.0     19.0      68
4.0       0.600000  0.400000       3.0      2.0       5
5.0       0.791667  0.208333      19.0      5.0      24
6.0       1.000000  0.000000       8.0      0.0       8
7.0       1.000000  0.000000       1.0      0.0       1


In [294]:
# '난자_혼합_경과일' 변수의 값이 6 이상인 행 제거
train = train[train['난자_혼합_경과일'] < 6]
test = test[test['난자_혼합_경과일'] < 6]

In [295]:
value_counts_ratio(train, '난자_혼합_경과일')


난자_혼합_경과일별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.722413  0.277587  145598  55946  201544
1.0       0.845679  0.154321     411     75     486
2.0       0.752475  0.247525      76     25     101
3.0       0.720588  0.279412      49     19      68
4.0       0.600000  0.400000       3      2       5
5.0       0.791667  0.208333      19      5      24


In [296]:
train['난자_혼합_경과일'].isnull().sum()

0

In [297]:
print("결측값 개수:", train['난자_혼합_경과일'].isnull().sum())

train['난자_혼합_경과일'].fillna(-1, inplace=True)
test['난자_혼합_경과일'].fillna(-1, inplace=True)

print("결측값 개수:", train['난자_혼합_경과일'].isnull().sum())

결측값 개수: 0
결측값 개수: 0


In [298]:
value_counts_ratio(train, '난자_혼합_경과일')


난자_혼합_경과일별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
0.0       0.722413  0.277587  145598  55946  201544
1.0       0.845679  0.154321     411     75     486
2.0       0.752475  0.247525      76     25     101
3.0       0.720588  0.279412      49     19      68
4.0       0.600000  0.400000       3      2       5
5.0       0.791667  0.208333      19      5      24


### 66. 배아_이식_경과일

첫 시점으로부터 배아 이식까지 경과한 일수

In [299]:
value_counts_ratio(train, '배아_이식_경과일')


배아_이식_경과일별 임신_성공_여부 비율 및 갯수

                 0         1      0      1  Total
variable                                         
0.0       0.757576  0.242424     25      8     33
1.0       0.868421  0.131579    462     70    532
2.0       0.789997  0.210003  25163   6689  31852
3.0       0.742306  0.257694  42065  14603  56668
4.0       0.650434  0.349566   1723    926   2649
5.0       0.595381  0.404619  48361  32866  81227
6.0       0.700000  0.300000   1925    825   2750
7.0       0.597561  0.402439     49     33     82


In [300]:
print("결측값 개수:", train['배아_이식_경과일'].isnull().sum())

train['배아_이식_경과일'].fillna(-1, inplace=True)
test['배아_이식_경과일'].fillna(-1, inplace=True)

print("결측값 개수:", train['배아_이식_경과일'].isnull().sum())

결측값 개수: 26435
결측값 개수: 0


In [301]:
value_counts_ratio(train, '배아_이식_경과일')


배아_이식_경과일별 임신_성공_여부 비율 및 갯수

                 0         1      0      1  Total
variable                                         
-1.0      0.998033  0.001967  26383     52  26435
 0.0      0.757576  0.242424     25      8     33
 1.0      0.868421  0.131579    462     70    532
 2.0      0.789997  0.210003  25163   6689  31852
 3.0      0.742306  0.257694  42065  14603  56668
 4.0      0.650434  0.349566   1723    926   2649
 5.0      0.595381  0.404619  48361  32866  81227
 6.0      0.700000  0.300000   1925    825   2750
 7.0      0.597561  0.402439     49     33     82


### 67. 배아_해동_경과일

첫 시점으로부터 배아 해동까지 경과한 일수

In [302]:
value_counts_ratio(train, '배아_해동_경과일')


배아_해동_경과일별 임신_성공_여부 비율 및 갯수

                 0         1      0     1  Total
variable                                        
0.0       0.777778  0.222222    7.0   2.0      9
1.0       0.823529  0.176471   28.0   6.0     34
2.0       0.835227  0.164773  147.0  29.0    176
3.0       0.888199  0.111801  143.0  18.0    161
4.0       1.000000  0.000000   11.0   0.0     11
5.0       0.795620  0.204380  109.0  28.0    137
6.0       0.827586  0.172414   24.0   5.0     29
7.0       1.000000  0.000000    3.0   0.0      3


In [303]:
# '배아_해동_경과일' 변수의 값이 7인 행 제거
train = train[train['배아_해동_경과일'] != 7]
test = test[test['배아_해동_경과일'] != 7]

In [304]:
print("결측값 개수:", train['배아_해동_경과일'].isnull().sum())

train['배아_해동_경과일'].fillna(-1, inplace=True)
test['배아_해동_경과일'].fillna(-1, inplace=True)

print("결측값 개수:", train['배아_해동_경과일'].isnull().sum())

결측값 개수: 201668
결측값 개수: 0


In [305]:
value_counts_ratio(train, '배아_해동_경과일')


배아_해동_경과일별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-1.0      0.722395  0.277605  145684.0  55984.0  201668
 0.0      0.777778  0.222222       7.0      2.0       9
 1.0      0.823529  0.176471      28.0      6.0      34
 2.0      0.835227  0.164773     147.0     29.0     176
 3.0      0.888199  0.111801     143.0     18.0     161
 4.0      1.000000  0.000000      11.0      0.0      11
 5.0      0.795620  0.204380     109.0     28.0     137
 6.0      0.827586  0.172414      24.0      5.0      29


### 68. 임신_성공_여부

해당 시술로부터 임신 성공 여부

In [306]:
train['임신_성공_여부'].value_counts()

임신_성공_여부
0    146153
1     56072
Name: count, dtype: int64

## 변수 상테 확인

In [307]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 202225 entries, 0 to 256349
Data columns (total 63 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ID                      202225 non-null  object 
 1   시술_당시_나이                202225 non-null  float64
 2   임신_시도_또는_마지막_임신_경과_연수   202225 non-null  float64
 3   특정_시술_유형                202223 non-null  object 
 4   배란_자극_여부                202225 non-null  int64  
 5   단일_배아_이식_여부             202225 non-null  float64
 6   착상_전_유전_검사_사용_여부        202225 non-null  float64
 7   착상_전_유전_진단_사용_여부        202225 non-null  float64
 8   남성_주_불임_원인              202225 non-null  int64  
 9   남성_부_불임_원인              202225 non-null  int64  
 10  여성_주_불임_원인              202225 non-null  int64  
 11  여성_부_불임_원인              202225 non-null  int64  
 12  부부_주_불임_원인              202225 non-null  int64  
 13  부부_부_불임_원인              202225 non-null  int64  
 14  불명확_불임_원인               2

In [308]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71350 entries, 0 to 90066
Data columns (total 62 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      71350 non-null  object 
 1   시술_당시_나이                71350 non-null  float64
 2   임신_시도_또는_마지막_임신_경과_연수   71350 non-null  float64
 3   특정_시술_유형                71350 non-null  object 
 4   배란_자극_여부                71350 non-null  int64  
 5   단일_배아_이식_여부             71350 non-null  float64
 6   착상_전_유전_검사_사용_여부        71350 non-null  float64
 7   착상_전_유전_진단_사용_여부        71350 non-null  float64
 8   남성_주_불임_원인              71350 non-null  int64  
 9   남성_부_불임_원인              71350 non-null  int64  
 10  여성_주_불임_원인              71350 non-null  int64  
 11  여성_부_불임_원인              71350 non-null  int64  
 12  부부_주_불임_원인              71350 non-null  int64  
 13  부부_부_불임_원인              71350 non-null  int64  
 14  불명확_불임_원인               71350 non-null  int

## 데이터 저장

In [309]:
# '임신_성공_여부' 변수의 값 개수 계산
value_counts = train['임신_성공_여부'].value_counts()

# 각 값이 전체에서 차지하는 비율 계산
value_percentages = value_counts / len(train) * 100

# 결과 출력
print("임신_성공_여부 값 개수:")
print(value_counts)
print("\n임신_성공_여부 값 비율 (%):")
print(value_percentages)

임신_성공_여부 값 개수:
임신_성공_여부
0    146153
1     56072
Name: count, dtype: int64

임신_성공_여부 값 비율 (%):
임신_성공_여부
0    72.272469
1    27.727531
Name: count, dtype: float64


In [134]:
# CSV 파일 저장
train.to_csv('../data/IVF_train_dataset_41.csv', index=False, encoding='utf-8-sig')
test.to_csv('../data/IVF_test_dataset_41.csv', index=False, encoding='utf-8-sig')

.